复现 [Chinchilla](https://arxiv.org/pdf/2203.15556.pdf) 中的一些缩放定律结果。虽然无法让数字完全匹配，但仍可用作粗略指南来帮助确定计算最优模型。还包含用于计算浮点运算和参数数量的相关工具。

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

## 参数

首先进行一些参数计算：

In [ ]:
def gpt_params(seq_len, vocab_size, d_model, num_heads, num_layers):
    """根据GPT配置计算总参数数量"""
    ffw_size = 4*d_model # 在GPT中，中间特征的数量始终是4*d_model
    # 令牌和位置嵌入
    embeddings = d_model * vocab_size + d_model * seq_len
    # transformer块
    attention = 3*d_model**2 + 3*d_model # 权重和偏置
    attproj = d_model**2 + d_model
    ffw = d_model*(ffw_size) + ffw_size
    ffwproj = ffw_size*d_model + d_model
    layernorms = 2*2*d_model
    # dense层
    ln_f = 2*d_model
    dense = d_model*vocab_size # 注意：这里没有偏置
    # 注意：嵌入不包含在参数计数中！
    total_params = num_layers*(attention + attproj + ffw + ffwproj + layernorms) + ln_f + dense
    return total_params

gpt2 = dict(seq_len = 1024, vocab_size = 50257, d_model = 768, num_heads = 12, num_layers = 12)
gpt_params(**gpt2)/1e6

OpenAI 报告 gpt2 (small) 有 124M 参数，所以这是匹配的。此外，将 OpenAI 权重加载到 nanoGPT 中然后调用 `model.parameters()` 完全匹配上述数字并验证了实现。现在是 Chinchilla 参数：

In [ ]:
def chinchilla_params(seq_len, vocab_size, d_model, num_heads, num_layers, ffw_size):
    """Chinchilla模型中的参数。与GPT不同，它们使用相对位置嵌入。"""
    # 仅令牌嵌入
    embeddings = d_model * vocab_size
    # transformer块
    attention = 3*d_model**2 + 3*d_model # 权重和偏置
    relative_pos = d_model**2 + 2*d_model # 相对键，内容偏置，相对偏置
    attproj = d_model**2 + d_model
    ffw = d_model*ffw_size + ffw_size
    ffwproj = ffw_size*d_model + d_model
    layernorms = 2*2*d_model
    # dense层
    ln_f = 2*d_model
    dense = d_model*vocab_size # 注意：这里没有偏置
    # 注意：嵌入不包含在参数计数中！
    total_params = num_layers*(attention + relative_pos + attproj + ffw + ffwproj + layernorms) + ln_f + dense
    return total_params


In [ ]:
# 加载论文最后一页中所有50个Chinchilla模型
import json
chinchilla_models_txt = '[[44000000.0, 512, 2048, 64, 8, 8], [57000000.0, 576, 2304, 64, 9, 9], [74000000.0, 640, 2560, 64, 10, 10], [90000000.0, 640, 2560, 64, 10, 13], [106000000.0, 640, 2560, 64, 10, 16], [117000000.0, 768, 3072, 64, 12, 12], [140000000.0, 768, 3072, 64, 12, 15], [163000000.0, 768, 3072, 64, 12, 18], [175000000.0, 896, 3584, 64, 14, 14], [196000000.0, 896, 3584, 64, 14, 16], [217000000.0, 896, 3584, 64, 14, 18], [251000000.0, 1024, 4096, 64, 16, 16], [278000000.0, 1024, 4096, 64, 16, 18], [306000000.0, 1024, 4096, 64, 16, 20], [425000000.0, 1280, 5120, 128, 10, 18], [489000000.0, 1280, 5120, 128, 10, 21], [509000000.0, 1408, 5632, 128, 11, 18], [552000000.0, 1280, 5120, 128, 10, 24], [587000000.0, 1408, 5632, 128, 11, 21], [632000000.0, 1536, 6144, 128, 12, 19], [664000000.0, 1408, 5632, 128, 11, 24], [724000000.0, 1536, 6144, 128, 12, 22], [816000000.0, 1536, 6144, 128, 12, 25], [893000000.0, 1792, 7168, 128, 14, 20], [1018000000.0, 1792, 7168, 128, 14, 23], [1143000000.0, 1792, 7168, 128, 14, 26], [1266000000.0, 2048, 8192, 128, 16, 22], [1424000000.0, 2176, 8704, 128, 17, 22], [1429000000.0, 2048, 8192, 128, 16, 25], [1593000000.0, 2048, 8192, 128, 16, 28], [1609000000.0, 2176, 8704, 128, 17, 25], [1731000000.0, 2304, 9216, 128, 18, 24], [1794000000.0, 2176, 8704, 128, 17, 28], [2007000000.0, 2304, 9216, 128, 18, 28], [2283000000.0, 2304, 9216, 128, 18, 32], [2298000000.0, 2560, 10240, 128, 20, 26], [2639000000.0, 2560, 10240, 128, 20, 30], [2980000000.0, 2560, 10240, 128, 20, 34], [3530000000.0, 2688, 10752, 128, 22, 36], [3802000000.0, 2816, 11264, 128, 22, 36], [4084000000.0, 2944, 11776, 128, 22, 36], [4516000000.0, 3072, 12288, 128, 24, 36], [6796000000.0, 3584, 14336, 128, 28, 40], [9293000000.0, 4096, 16384, 128, 32, 42], [11452000000.0, 4352, 17408, 128, 32, 47], [12295000000.0, 4608, 18432, 128, 36, 44], [12569000000.0, 4608, 18432, 128, 32, 47], [13735000000.0, 4864, 19456, 128, 32, 47], [14940000000.0, 4992, 19968, 128, 32, 49], [16183000000.0, 5120, 20480, 128, 40, 47]]'
chilchilla_models = json.loads(chinchilla_models_txt) # 所有50个模型
chilchilla_models[0] # 表A9中的参数、d_model、ffw_size、kv_size、n_heads、n_layers的元组

In [ ]:
for m in chilchilla_models[-5:]: # 只打印表格的最后5个模型
    p, d, f, k, h, l = m
    nparams = chinchilla_params(seq_len = 1024, vocab_size = 32000, d_model = d, num_heads = h, num_layers = l, ffw_size=f)
    print(f"我们估算的参数: {nparams/1e6:.4f}M, chinchilla参数: {p/1e6:.4f}M, d_model: {d}, n_heads: {h}, n_layers: {l}")

我们几乎能够复现Chinchilla模型的参数计数。

现在转向FLOPs：

## 浮点运算

In [ ]:
def chinchilla_flops(seq_len, vocab_size, d_model, num_heads, num_layers, ffw_size):
    """ 
    计算总FLOPs数量，参见Chinchilla论文附录F作为参考：
    https://arxiv.org/pdf/2203.15556.pdf
    """ 
    key_size = d_model // num_heads

    # 嵌入
    embeddings = 2 * seq_len * vocab_size * d_model

    # 注意力
    # 键、查询、值投影
    attention = 2 * 3 * seq_len * d_model * (key_size * num_heads)
    # key @ query logits
    attlogits = 2 * seq_len * seq_len * (key_size * num_heads)
    # softmax
    attsoftmax = 3 * num_heads * seq_len * seq_len # 3* 是为了减法(max)、exp、除法(?)
    # softmax @ value 归约
    attvalue = 2 * seq_len * seq_len * (key_size * num_heads)
    # 最终线性层
    attlinear = 2 * seq_len * (key_size * num_heads) * d_model
    att = attention + attlogits + attsoftmax + attvalue + attlinear
    # 前馈层
    dense = 2 * seq_len * (d_model * ffw_size + d_model * ffw_size)

    # logits
    logits = 2 * seq_len * d_model * vocab_size
    
    # 这是你所期望的：
    # forward_flops = embeddings + num_layers * (att + dense) + logits
    # 但是：
    # 根据作者通信，论文中显然有一个错误，
    # 他们没有计算嵌入和logits来复现表4。所以改为：
    forward_flops = num_layers * (att + dense)
    backward_flops = 2 * forward_flops # 如Kaplan等人2020年所述
    total_flops = forward_flops + backward_flops

    return total_flops

In [ ]:
# 现在尝试复现Chinchilla论文附录中的表A4，
# 将上面准确的flops与近似flops F = 6*N*D进行比较
# 注意Chinchilla提到使用vocab_size = 32K

chilchilla_models_table4 = [
  [10, 640, 2560, 10, 64],
  [20, 1024, 4096, 16, 64],
  [24, 1280, 5120, 10, 128 ],
  [26, 1792, 7168, 14, 128 ],
  [28, 2048, 8192, 16, 128],
  [40,  3584, 14336, 28, 128]
]

rows = []
for num_layers, d_model, ffw_size, num_heads, _ in chilchilla_models_table4:

    args = dict(seq_len = 2048, vocab_size = 32000, d_model = d_model, 
                num_heads = num_heads, num_layers = num_layers, ffw_size=ffw_size)

    D = args['seq_len'] # 数据集大小（无论如何都会抵消，用于下面的比率计算）
    N = chinchilla_params(**args)
    F = chinchilla_flops(**args)

    approx_flops = 6*D*N # 近似flops
    chinch_flops = F * (float(D) / args['seq_len']) # 根据Chinchilla论文计算的精确flops

    # print('---')
    # print(f"params: {N/1e6:.2f}M")
    # print(f"approx flops: {approx_flops/1e9:.2f}B")
    # print(f"chinchilla flops: {chinch_flops/1e9:.2f}B")
    # print(f"ratio (chinchilla / approx): {chinch_flops / approx_flops:.2f}")

    # 首先从args复制所有键值到out
    out = {k:v for k,v in args.items()}
    # 然后添加计算的值
    out['N'] = N
    out['F'] = F
    out['approx_flops'] = approx_flops
    out['chinch_flops'] = chinch_flops
    out['ratio'] = chinch_flops / approx_flops
    rows.append(out)

# 从rows创建pandas数据框
df = pd.DataFrame(rows)
df

相当匹配！除了参数计数仍然不是完全准确。

## 缩放定律：方法3

在他们的"方法3"中，Chinchilla论文拟合了一个函数L(N,D)来近似最终损失，给定模型大小和数据大小。这是最终拟合：

In [ ]:
def L(N, D):
    """ 
    根据Chinchilla论文，给定N个参数和D个数据集大小（以令牌为单位）来近似损失。
    """
    E = 1.69 # 自然语言的熵，无限模型在无限数据上的极限
    A = 406.4
    B = 410.7
    alpha = 0.34
    beta = 0.28
    return A / (N ** alpha) + B / (D ** beta) + E

ns = 10 ** np.arange(7, 11, step=2**-4) # 模型大小从10M到100B
ds = 10 ** np.arange(9, 12, step=2**-4) # 数据集大小从1B到1T
plt.figure(figsize=(12, 5))
plt.subplot(121)
# 创建损失L的2D等高线图，作为ns,ds中模型大小和数据集大小的函数
loss2d = np.log10(np.array([[L(n, d) for d in ds] for n in ns]))
plt.imshow(loss2d, extent=[9, 12, 7, 11], origin='lower', alpha=0.5)
plt.contour(loss2d, levels=30, extent=[9, 12, 7, 11], origin='lower')
plt.xlabel('log10(数据集大小)')
plt.ylabel('log10(模型大小)')
plt.title('损失')
plt.colorbar()
# 绘制每个点的计算量，这是一个确定性函数：flops = 6*N*D
plt.subplot(122)
compute2d = np.log10(np.array([[6*n*d for d in ds] for n in ns]))
plt.imshow(compute2d, extent=[9, 12, 7, 11], origin='lower', alpha=0.5)
plt.contour(compute2d, levels=30, extent=[9, 12, 7, 11], origin='lower')
plt.xlabel('log10(数据集大小)')
plt.ylabel('log10(模型大小)')
plt.title('log10 flops')
plt.colorbar()

好的，所以给定任何N,D我们可以估算两者：1) 损失，和 2) 总flops。现在我们想解决以下问题：给定特定的flops预算C，求：N_opt, D_opt = argmin_{FLOPs(N,D) = C} L(N, D)。即我们应该训练多大的模型以及训练多少令牌？

In [ ]:
c = 2.21e19 # 目标计算预算（通常知道这个是因为我们知道多少GPU运行多长时间）
# （我从表A3的第1行获得了这个flop数字）
# 扫描模型大小从10M到100B
ns = 10 ** np.arange(7, 11, step=2**-4)
# 使用 C = 6*N*D，求解维持计算预算c的D
ds = c / (6 * ns)
# 评估每种情况下的损失
losses = L(ns, ds)
# 找到argmin
best = np.argmin(losses)
print(f"最佳模型大小: {ns[best]/1e6:.2f}M")
print(f"最佳数据集大小: {ds[best]/1e9:.2f}B")
# 绘制损失
plt.figure(figsize=(3,3))
plt.plot(ns, losses)
plt.xscale('log')
# 在最佳模型大小处绘制垂直线
plt.axvline(ns[best], color='red')
plt.xlabel('模型大小')
plt.ylabel('损失')

在上图中，基本上最佳左侧的模型太小且训练时间过长。最佳右侧的模型太大且训练时间过短。红线处的模型恰到好处。

现在，Chinchilla论文说这个flop预算的最佳模型大小是400M参数和9.2B令牌（而不是316M参数和11.65B令牌），所以这里也存在一些未解决的差异...

In [ ]:
# 计算一系列计算预算的Chinchilla最优模型

# 扫描计算预算从1e17到1e26
cs = 10 ** np.arange(17, 26, step=2**-8)
models = []
for c in cs:
    # 扫描模型大小
    ns = 10 ** np.arange(7, 14, step=2**-8)
    # 维持给定计算预算的数据集大小
    ds = c / (6 * ns)
    # 每个点的损失
    losses = L(ns, ds)
    # 最佳模型的n,d
    best = np.argmin(losses)
    models.append((c, ns[best], ds[best])) # c, n, d 元组日志

len(models)

In [ ]:
query_model_size = 400e6
ns = np.array([n for c, n, d in models])
ds = np.array([d for c, n, d in models])
# 在ns中找到最接近模型大小的索引
ix = np.argmin(np.abs(ns - query_model_size))
# 检索相应的参数、flops和数据大小
print("找到的最接近模型:")
print(f"模型大小: {ns[ix]/1e6:.2f}M")
print(f"数据集大小: {ds[ix]/1e9:.2f}B")
print(f"flops: {6*ns[ix]*ds[ix]:e}")
print(f"损失: {L(ns[ix], ds[ix]):.2f}")

根据我对Chinchilla论文表A3的理解，这应该是9.2B。

## 缩放定律：方法2

方法2可能是我最喜欢的一个，因为它固定了flop预算并运行多个模型/数据集大小，测量损失，拟合抛物线，并得到最小值。所以这是对我们所追求的相当直接的测量。然后，计算任何给定模型大小的计算最优令牌数的最佳方法是通过简单插值。

In [ ]:
# 方法1的数据
# # 参数, 令牌
# raw = [
#     [400e6, 8e9],
#     [1e9, 20.2e9],
#     [10e9, 205.1e9],
#     [67e9, 1.5e12],
#     [175e9, 3.7e12],
#     [280e9, 5.9e12],
#     [520e9, 11e12],
#     [1e12, 21.2e12],
#     [10e12, 216.2e12],
# ]

# 方法2的数据
# 参数, 令牌
raw = [
    [400e6, 7.7e9],
    [1e9, 20.0e9],
    [10e9, 219.5e9],
    [67e9, 1.7e12],
    [175e9, 4.3e12],
    [280e9, 7.1e12],
    [520e9, 13.4e12],
    [1e12, 26.5e12],
    [10e12, 292.0e12],
]

In [ ]:
# 通过线性回归拟合原始数据的直线
import numpy as np
x = np.array([np.log10(x[0]) for x in raw])
y = np.array([np.log10(x[1]) for x in raw])
A = np.vstack([x, np.ones(len(x))]).T
m, c = np.linalg.lstsq(A, y, rcond=None)[0]
print(f"y = {m}x + {c}")

In [ ]:
plt.figure(figsize=(3, 3))
# 绘制直线
plt.plot([q[0] for q in raw], [10**(m*np.log10(q[0]) + c) for q in raw], label='线性回归', color='r')
# 绘制原始数据
plt.scatter([q[0] for q in raw], [q[1] for q in raw], label='原始数据')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('参数')
plt.ylabel('令牌')
plt.title('计算最优模型')
plt.grid()

In [ ]:
xquery = 124e6 # 在此查询模型大小（例如GPT-2 small是124M）
yquery = 10**(m*np.log10(xquery) + c)
print(f"预测{xquery:e}令牌的参数: {yquery:e}")